### Extract text from Pdf documents and apply NER

In [252]:
import pdfplumber, os
import os.path as osp
import pandas as pd
from pathlib import Path
from pdf_extract.services import file
from pdf_extract.config import global_config as glob
from pdf_extract.config import config
from pdf_extract.utils import utils
from importlib import reload
import fasttext

reload(glob)
reload(config)
reload(utils)
reload(file)

<module 'pdf_extract.services.file' from '/home/alexv84/Documents/GitHub/pdf_extract/src/pdf_extract/services/file.py'>

In [253]:
list_of_NEG_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'negatives'))
n_neg = len(list_of_NEG_docs)

list_of_POS_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'positives'))
n_pos = len(list_of_POS_docs)

print(n_pos)
print(n_neg)

13
13


Read in all Pdfs in directory:

Postives only:

In [254]:
raw_corpus1 = pd.DataFrame(columns=['text', 'fname'])

for z, fname in enumerate(list_of_POS_docs):
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"positives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            #print(pdf.metadata)
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            #print(f"Page {i}")
            #print(page.extract_text())
            i += 1
    
    raw_corpus1.loc[z] = [text,fname]  
raw_corpus1['label'] = 'positive'

Negatives only:

In [255]:
raw_corpus2 = pd.DataFrame(columns=['text', 'fname'])

for z, fname in enumerate(list_of_NEG_docs):
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"negatives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            i += 1
    
    raw_corpus2.loc[z] = [text,fname]   
raw_corpus2['label'] = 'negative'

In [256]:
raw_corpus = pd.concat([raw_corpus1,raw_corpus2], axis=0, ignore_index=True)
#raw_corpus

In [257]:
#page_objects

#page.extract_words()

In [258]:
#page_instance = page_objects['1']

In [259]:
#pdf.metadata
#page_instance

In [271]:
from sklearn.model_selection import train_test_split

reload(utils)

#X_train, X_valid, X_test, y_train, y_valid, y_test = utils.train_test_split_extend(X['text_cleaned'], y=X['label'], test_size=[0.2, 0.2], random_state=42, shuffle=True)
X_train, X_test, y_train, y_test = utils.train_test_split_extend(raw_corpus, y=raw_corpus['label'], test_size=[0.3], random_state=123, shuffle=True)

print(X_train.shape)
#print(y_train.shape)
print(X_test.shape)
#print(X_valid.shape)


(18, 3)
(8, 3)


Preprocess corpus:

In [273]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer #, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from pdf_extract.resources import preprocessor as preproc

reload(preproc)

# Preprocess corpus:
cleaner = preproc.clean_text(language='english', lemma = False, stem = True)

# Train
X_train_cleaned = cleaner.fit_transform(X_train['text'])

# Test
X_test_cleaned = cleaner.transform(X_test['text'])


Using english language.
Using 179 stop words.
Added 0 stopword(s).
Adding custom stop words...
Loading nltk stemmer.
Setting to lower cases.
Removing whitespaces.
Applying word tokenizer.
Removing custom stopwords.
Removing punctuations.
Removing numbers.
Removing digits.
Removing non-alphabetic characters.
Removing short tokens.
Applying stemming.
Finished preprocessing!
Setting to lower cases.
Removing whitespaces.
Applying word tokenizer.
Removing custom stopwords.
Removing punctuations.
Removing numbers.
Removing digits.
Removing non-alphabetic characters.
Removing short tokens.
Applying stemming.
Finished preprocessing!


[nltk_data] Downloading package punkt to /home/alexv84/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexv84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Train model:

In [274]:
parameters = {
    'vectorizer__max_df': (0.5, 0.75, 1.0),
     'vectorizer__max_features': (None, 500, 1000, 5000, 10000),
     #'vectorizer__analyzer' : ("word", "char", "char_wb"),
    'vectorizer__ngram_range': ((2, 2), (1, 3), (2, 3), (3, 3), (3, 5))  # unigrams or bigrams
}


pipeline = Pipeline([
   #('cleaner', preproc.clean_text(language='english', lemma = False, stem = False)),
#    ('vectorizer', CountVectorizer(lowercase=True, 
#                        #token_pattern = '(?u)(?:(?!\d)\w)+\\w+', 
#                        #analyzer = 'char_wb', 
#                        tokenizer = None, stop_words = None)), 
    ('vectorizer', TfidfVectorizer(lowercase=True, norm="l1",
                       #token_pattern = '(?u)(?:(?!\d)\w)+\\w+', 
                       analyzer = 'char_wb', 
                       tokenizer = None, stop_words = None)), 
   ('scaler', StandardScaler(with_mean=False)),
   ('model', MultinomialNB())
])

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

gs = grid_search.fit(X_train_cleaned, y_train)

best_parameters = grid_search.best_estimator_.get_params()
# best_model refit:
trained = grid_search.best_estimator_


Fitting 5 folds for each of 75 candidates, totalling 375 fits


In [275]:
#-------------
# Save model:
#-------------
file.PickleService(path="trained_model.pkl", root_path=glob.UC_DATA_DIR, is_df=False).doWrite(X=trained)


Pickle Service Read from file: /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/trained_model.pkl


In [276]:
# Predict:
y_pred = trained.predict(X_train_cleaned)

#print(confusion_matrix(test_set['label'], y_pred))
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

    negative       0.90      1.00      0.95         9
    positive       1.00      0.89      0.94         9

    accuracy                           0.94        18
   macro avg       0.95      0.94      0.94        18
weighted avg       0.95      0.94      0.94        18



In [277]:
# Predict:
y_pred = trained.predict(X_test_cleaned)

#print(confusion_matrix(test_set['label'], y_pred))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.75      0.75      0.75         4
    positive       0.75      0.75      0.75         4

    accuracy                           0.75         8
   macro avg       0.75      0.75      0.75         8
weighted avg       0.75      0.75      0.75         8



In [289]:
X_test['Prob. positive class'] = trained.predict_proba(X_test_cleaned)[:,1]

#X_test.sort_values(by = 'Prob. positive class', ascending=False)

In [288]:
X_train['Prob. positive class'] = trained.predict_proba(X_train_cleaned)[:,1]

#X_train.sort_values(by = 'Prob. positive class', ascending=False)